In [34]:
import torch
from torch.serialization import safe_globals

from esm.models.esm3 import ESM3
from esm.tokenization.sequence_tokenizer import EsmSequenceTokenizer

In [35]:
# Register the ESM3 class as safe for unpickling.
torch.serialization.add_safe_globals([ESM3])

In [7]:
https://github.com/evolutionaryscale/esm/issues/178

SyntaxError: invalid syntax (2229619655.py, line 1)

In [36]:
backbone_save_path = "/home/jupyter/DATA/evqlv-dev/model-weights/esm3_backbone/esm3_backbone_model.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
with safe_globals([("esm.models.esm3.ESM3", ESM3)]):
    loaded_backbone_model = torch.load(backbone_save_path, map_location=device, weights_only=False)
    loaded_backbone_model.eval()

cuda


In [37]:
state_dict = loaded_backbone_model.state_dict()
for key, tensor in state_dict.items():
    print(f"{key}: {tensor.shape}")
    
total_params = sum(p.numel() for p in loaded_backbone_model.parameters())
print("Total number of parameters:", total_params)

encoder.sequence_embed.weight: torch.Size([64, 1536])
encoder.plddt_projection.weight: torch.Size([1536, 16])
encoder.plddt_projection.bias: torch.Size([1536])
encoder.structure_per_res_plddt_projection.weight: torch.Size([1536, 16])
encoder.structure_per_res_plddt_projection.bias: torch.Size([1536])
encoder.structure_tokens_embed.weight: torch.Size([4101, 1536])
encoder.ss8_embed.weight: torch.Size([11, 1536])
encoder.sasa_embed.weight: torch.Size([19, 1536])
encoder.function_embed.0.weight: torch.Size([260, 192])
encoder.function_embed.1.weight: torch.Size([260, 192])
encoder.function_embed.2.weight: torch.Size([260, 192])
encoder.function_embed.3.weight: torch.Size([260, 192])
encoder.function_embed.4.weight: torch.Size([260, 192])
encoder.function_embed.5.weight: torch.Size([260, 192])
encoder.function_embed.6.weight: torch.Size([260, 192])
encoder.function_embed.7.weight: torch.Size([260, 192])
encoder.residue_embed.weight: torch.Size([1478, 1536])
transformer.blocks.0.attn.layern

# Validate encoder is working

In [6]:
from esm.tokenization.sequence_tokenizer import EsmSequenceTokenizer


In [67]:
# for ESM3
# Initialize sequence 
seq_tokenizer = EsmSequenceTokenizer()
test_sequence = "CSSDGSYGFGAMDYW"
seq_tokens = seq_tokenizer.encode(test_sequence)
seq_tokens_tensor = torch.tensor(seq_tokens, dtype=torch.int64).unsqueeze(0).to(device)

# Create required tensors
# dummy_average_plddt = torch.ones(seq_tokens_tensor.shape, dtype=torch.float32, device=device)
# dummy_per_res_plddt = torch.ones(seq_tokens_tensor.shape, dtype=torch.float32, device=device)
# dummy_structure_tokens = torch.zeros(seq_tokens_tensor.shape, dtype=torch.int64, device=device)

# Run forward pass
with torch.no_grad():
    embeddings = loaded_backbone_model.encoder.sequence_embed(seq_tokens_tensor)
    print("Embeddings shape:", embeddings.shape)
    
    
# validate other embedding types work (structure, etc. for ESM3)
    
    

Embeddings shape: torch.Size([1, 17, 1536])


In [49]:
loaded_backbone_model.encoder.

EncodeInputs(
  (sequence_embed): Embedding(64, 1536)
  (plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
  (structure_per_res_plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
  (structure_tokens_embed): Embedding(4101, 1536)
  (ss8_embed): Embedding(11, 1536)
  (sasa_embed): Embedding(19, 1536)
  (function_embed): ModuleList(
    (0-7): 8 x Embedding(260, 192, padding_idx=0)
  )
  (residue_embed): EmbeddingBag(1478, 1536, mode='sum', padding_idx=0)
)

In [56]:
import torch
from esm.tokenization.sequence_tokenizer import EsmSequenceTokenizer
from esm.models.esm3 import ESM3

In [63]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [69]:
seq_tokenizer = EsmSequenceTokenizer()
test_sequence = "CSSDGSYGFGAMDYW"
seq_tokens = seq_tokenizer.encode(test_sequence)
seq_tokens_tensor = torch.tensor(seq_tokens, dtype=torch.int64).unsqueeze(0).to(device)

In [86]:
dummy_structure_tokens = torch.zeros_like(seq_tokens_tensor, dtype=torch.int64, device=device)

# Dummy tokens for ss8 (secondary structure, 11 classes) and SASA (solvent accessibility, 19 classes)
dummy_ss8_tokens = torch.zeros_like(seq_tokens_tensor, dtype=torch.int64, device=device)
dummy_sasa_tokens = torch.zeros_like(seq_tokens_tensor, dtype=torch.int64, device=device)

# Dummy pLDDT values (typically float values per residue)
dummy_average_plddt = torch.ones_like(seq_tokens_tensor, dtype=torch.float32, device=device)
dummy_per_res_plddt = torch.ones_like(seq_tokens_tensor, dtype=torch.float32, device=device)

batch_size, seq_len = seq_tokens_tensor.shape
dummy_rbf = torch.ones(batch_size, seq_len, 16,
                       dtype=loaded_backbone_model.encoder.plddt_projection.weight.dtype,
                       device=device)


In [92]:
dummy_structure_tokens

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [94]:
structure_embedding.shape

torch.Size([1, 17, 1536])

In [88]:
with torch.no_grad():
    # 1. Sequence embedding:
    seq_embedding = loaded_backbone_model.encoder.sequence_embed(seq_tokens_tensor)
    print("Sequence embedding shape:", seq_embedding.shape)
    
    # 2. Structure tokens embedding:
    structure_embedding = loaded_backbone_model.encoder.structure_tokens_embed(dummy_structure_tokens)
    print("Structure tokens embedding shape:", structure_embedding.shape)
    
    # 3. pLDDT projection:
    plddt_embedding = loaded_backbone_model.encoder.plddt_projection(dummy_rbf)
    print("pLDDT projection output shape:", plddt_embedding.shape)
    
    # 4. Structure per-res pLDDT projection:
    structure_plddt_embedding = loaded_backbone_model.encoder.structure_per_res_plddt_projection(dummy_rbf)
    print("Structure per-res pLDDT projection output shape:", structure_plddt_embedding.shape)
    
    # 5. SS8 embedding:
    ss8_embedding = loaded_backbone_model.encoder.ss8_embed(dummy_ss8_tokens)
    print("SS8 embedding shape:", ss8_embedding.shape)
    
    # 6. SASA embedding:
    sasa_embedding = loaded_backbone_model.encoder.sasa_embed(dummy_sasa_tokens)
    print("SASA embedding shape:", sasa_embedding.shape)
    
    # 7. Function embeddings (a ModuleList of 8 embeddings, each is an independent layer):
    for i, func_embed in enumerate(loaded_backbone_model.encoder.function_embed):
        # Here we simply use the same dummy sequence tokens for input.
        func_embedding = func_embed(seq_tokens_tensor)
        print(f"Function embedding {i} shape:", func_embedding.shape)
    
    # 8. Residue annotation embedding using EmbeddingBag:
    # EmbeddingBag expects a 1D tensor of indices and an offsets tensor.
    dummy_residue_tokens = torch.zeros(seq_tokens_tensor.shape, dtype=torch.int64, device=device)
    dummy_residue_tokens_flat = dummy_residue_tokens.view(-1)
    offsets = torch.tensor([0], dtype=torch.int64, device=device)
    residue_embedding = loaded_backbone_model.encoder.residue_embed(dummy_residue_tokens_flat, offsets)
    print("Residue embedding shape:", residue_embedding.shape)


Sequence embedding shape: torch.Size([1, 17, 1536])
Structure tokens embedding shape: torch.Size([1, 17, 1536])
pLDDT projection output shape: torch.Size([1, 17, 1536])
Structure per-res pLDDT projection output shape: torch.Size([1, 17, 1536])
SS8 embedding shape: torch.Size([1, 17, 1536])
SASA embedding shape: torch.Size([1, 17, 1536])
Function embedding 0 shape: torch.Size([1, 17, 192])
Function embedding 1 shape: torch.Size([1, 17, 192])
Function embedding 2 shape: torch.Size([1, 17, 192])
Function embedding 3 shape: torch.Size([1, 17, 192])
Function embedding 4 shape: torch.Size([1, 17, 192])
Function embedding 5 shape: torch.Size([1, 17, 192])
Function embedding 6 shape: torch.Size([1, 17, 192])
Function embedding 7 shape: torch.Size([1, 17, 192])
Residue embedding shape: torch.Size([1, 1536])


In [89]:
dummy_sasa_tokens

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [91]:
len(dummy_sasa_tokens[0])

17

# ESM3 Use PDB File

# Structure Tokenization

In [111]:
import torch
import numpy as np
from Bio.PDB import *
from scipy.spatial.transform import Rotation

def get_backbone_frames(coords):
    """Get local backbone frames from atomic coordinates"""
    n_atoms = coords.shape[0]
    
    # Calculate local frame using N, CA, C atoms
    n_ca = coords[:, 1] - coords[:, 0]  # N->CA vectors
    ca_c = coords[:, 2] - coords[:, 1]  # CA->C vectors
    
    # Normalize vectors
    n_ca = n_ca / torch.norm(n_ca, dim=-1, keepdim=True)
    ca_c = ca_c / torch.norm(ca_c, dim=-1, keepdim=True)
    
    # Calculate frame axes
    t = ca_c
    n = torch.cross(n_ca, ca_c)
    n = n / torch.norm(n, dim=-1, keepdim=True)
    b = torch.cross(t, n)
    
    # Stack into rotation matrices
    frames = torch.stack([n, b, t], dim=-1)
    return frames

def get_relative_orientations(frames):
    """Get relative orientations between backbone frames"""
    n_frames = frames.shape[0]
    
    # Calculate relative rotations between all pairs
    rel_rots = torch.matmul(frames.unsqueeze(1), frames.transpose(-2, -1).unsqueeze(0))
    
    # Convert to quaternions
    rel_quats = torch.zeros((n_frames, n_frames, 4), device=frames.device)
    for i in range(n_frames):
        for j in range(n_frames):
            r = Rotation.from_matrix(rel_rots[i,j].cpu().numpy())
            rel_quats[i,j] = torch.tensor(r.as_quat(), device=frames.device)
            
    return rel_quats

def structure_to_tokens(coords, max_radius=20.0, n_bins=50):
    """Convert structure to ESM tokens"""
    # Get CA distances
    ca_coords = coords[:, 1]  # CA atoms
    dists = torch.cdist(ca_coords, ca_coords)
    
    # Get orientations
    frames = get_backbone_frames(coords)
    orientations = get_relative_orientations(frames)
    
    # Discretize distances
    dist_bins = torch.linspace(0, max_radius, n_bins, device=coords.device)
    dist_tokens = torch.bucketize(dists.flatten(), dist_bins).reshape(dists.shape)
    
    # Discretize orientations using quaternion bins
    quat_bins = 8  # Number of bins per quaternion component
    quat_tokens = torch.floor((orientations + 1) * quat_bins/2).long()
    quat_tokens = quat_tokens[..., 0] * quat_bins**3 + \
                  quat_tokens[..., 1] * quat_bins**2 + \
                  quat_tokens[..., 2] * quat_bins + \
                  quat_tokens[..., 3]
                  
    # Combine distance and orientation tokens
    structure_tokens = dist_tokens * quat_bins**4 + quat_tokens
    
    return structure_tokens

# Load and process structure
parser = MMCIFParser()
with gzip.open('/home/jupyter/dan/data/1bey.cif.gz', 'rt') as f:
    structure = parser.get_structure('1BEY', f)

# Extract backbone coordinatemms
coords = []
for model in structure:
    for chain in model:
        backbone = []
        for residue in chain:
            if all(atom in residue for atom in ['N', 'CA', 'C']):
                backbone.append([
                    residue['N'].get_coord(),
                    residue['CA'].get_coord(),
                    residue['C'].get_coord()
                ])
        coords.append(backbone)

coords = torch.tensor(coords[0], dtype=torch.float32).to(device)

# Generate structure tokens
structure_tokens = structure_to_tokens(coords)
print(f"Structure tokens shape: {structure_tokens.shape}")

Structure tokens shape: torch.Size([214, 214])


structure_embedding = loaded_backbone_model.encoder.structure_tokens_embed(structure_tokens)
print("Structure tokens embedding shape:", structure_embedding.shape)
    

# objectives:
# 1. validate other embeddings work (may need antibody pdbs as input)
# 2. validate embedding dimensions / shape
# 3. Run multiple sequences in one pass?
# 4. Speed test 10, 100, 1000 sequences runtime
# 5. Training...OPT 3 guide a fine-tuning of this model (on 5 random antibody sequences)
# 6. LORA for training econo.y








